In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib


# Load the datasets
train_df = pd.read_csv("C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\train_data.txt", sep=' ::: ', engine='python')
test_df = pd.read_csv("C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\test_data.txt", sep=' ::: ', engine='python')
test_solution_df = pd.read_csv("C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\test_data_solution.txt", sep=' ::: ', engine='python')

# Display the first few rows of the datasets
train_df.head(), test_df.head(), test_solution_df.head()


(   1      Oscar et la dame rose (2009)        drama  \
 0  2                      Cupid (1997)     thriller   
 1  3  Young, Wild and Wonderful (1980)        adult   
 2  4             The Secret Sin (1915)        drama   
 3  5            The Unrecovered (2007)        drama   
 4  6            Quality Control (2011)  documentary   
 
   Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.  
 0  A brother and sister with a past incestuous re...                                                          

In [2]:
# Load training data into a dataframe
train_data_file = "C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\train_data.txt"
train_df = pd.read_csv(train_data_file, sep=' ::: ', header=None, engine='python')

# Assign column names
train_df.columns = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']

# Display the dataframe
print("Training Dataset:")
print(train_df.head())



Training Dataset:
   ID                             TITLE     GENRE  \
0   1      Oscar et la dame rose (2009)     drama   
1   2                      Cupid (1997)  thriller   
2   3  Young, Wild and Wonderful (1980)     adult   
3   4             The Secret Sin (1915)     drama   
4   5            The Unrecovered (2007)     drama   

                                         DESCRIPTION  
0  Listening in to a conversation between his doc...  
1  A brother and sister with a past incestuous re...  
2  As the bus empties the students for their fiel...  
3  To help their unemployed father make ends meet...  
4  The film's title refers not only to the un-rec...  


In [3]:
# Load test data into a dataframe
test_data_file = "C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\test_data.txt"
test_df = pd.read_csv(test_data_file, sep=' ::: ', header=None, engine='python')

# Assign column names
test_df.columns = ['ID', 'TITLE', 'DESCRIPTION']

# Display the dataframe
print("\nTest Dataset:")
print(test_df.head())



Test Dataset:
   ID                        TITLE  \
0   1         Edgar's Lunch (1998)   
1   2     La guerra de papá (1977)   
2   3  Off the Beaten Track (2010)   
3   4       Meu Amigo Hindu (2015)   
4   5            Er nu zhai (1955)   

                                         DESCRIPTION  
0  L.R. Brane loves his life - his car, his apart...  
1  Spain, March 1964: Quico is a very naughty chi...  
2  One year in the life of Albin and his family o...  
3  His father has died, he hasn't spoken with his...  
4  Before he was known internationally as a marti...  


In [4]:
# Load test data solutions into a dataframe
test_solution_data_file = "C:\\Users\\harih\\OneDrive\\Desktop\\CODSOFT\\TASK-1 MOVIE\\Genre Classification Dataset\\test_data_solution.txt"
test_solution_df = pd.read_csv(test_solution_data_file, sep=' ::: ', header=None, engine='python')

# Assign column names
test_solution_df.columns = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']

# Display the dataframe
print("\nTest Data Solutions Dataset:")
print(test_solution_df.head())



Test Data Solutions Dataset:
   ID                        TITLE        GENRE  \
0   1         Edgar's Lunch (1998)     thriller   
1   2     La guerra de papá (1977)       comedy   
2   3  Off the Beaten Track (2010)  documentary   
3   4       Meu Amigo Hindu (2015)        drama   
4   5            Er nu zhai (1955)        drama   

                                         DESCRIPTION  
0  L.R. Brane loves his life - his car, his apart...  
1  Spain, March 1964: Quico is a very naughty chi...  
2  One year in the life of Albin and his family o...  
3  His father has died, he hasn't spoken with his...  
4  Before he was known internationally as a marti...  


In [5]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = text.lower().strip()  # Convert to lowercase and strip whitespace
    tokens = text.split()  # Tokenize
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Lemmatize and remove stopwords
    return ' '.join(tokens)
train_df['clean_description'] = train_df['DESCRIPTION'].apply(preprocess_text)
test_df['clean_description'] = test_df['DESCRIPTION'].apply(preprocess_text)
test_solution_df['clean_description'] = test_solution_df['DESCRIPTION'].apply(preprocess_text)
train_df[['DESCRIPTION', 'clean_description']].head(), test_df[['DESCRIPTION', 'clean_description']].head(), test_solution_df[['DESCRIPTION', 'clean_description']].head()


(                                         DESCRIPTION  \
 0  Listening in to a conversation between his doc...   
 1  A brother and sister with a past incestuous re...   
 2  As the bus empties the students for their fiel...   
 3  To help their unemployed father make ends meet...   
 4  The film's title refers not only to the un-rec...   
 
                                    clean_description  
 0  listening conversation doctor parent 10 year o...  
 1  brother sister past incestuous relationship cu...  
 2  bus empty student field trip museum natural hi...  
 3  help unemployed father make end meet edith twi...  
 4  film title refers un recovered body ground zer...  ,
                                          DESCRIPTION  \
 0  L.R. Brane loves his life - his car, his apart...   
 1  Spain, March 1964: Quico is a very naughty chi...   
 2  One year in the life of Albin and his family o...   
 3  His father has died, he hasn't spoken with his...   
 4  Before he was known internatio

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train_df['clean_description']).toarray()
y_train = train_df['GENRE']
X_test = tfidf_vectorizer.transform(test_df['clean_description']).toarray()
X_train.shape, X_test.shape


((54214, 5000), (54200, 5000))

In [7]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    
}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    print(f"{model_name} trained successfully.")


Naive Bayes trained successfully.
Logistic Regression trained successfully.


In [8]:

from sklearn.metrics import accuracy_score, classification_report
y_test = test_solution_df['GENRE']
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy}")
    print(report)


c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Naive Bayes - Accuracy: 0.5206088560885609
              precision    recall  f1-score   support

      action       0.58      0.10      0.17      1314
       adult       0.49      0.08      0.14       590
   adventure       0.81      0.09      0.16       775
   animation       0.00      0.00      0.00       498
   biography       0.00      0.00      0.00       264
      comedy       0.52      0.43      0.47      7446
       crime       0.00      0.00      0.00       505
 documentary       0.57      0.87      0.69     13096
       drama       0.46      0.82      0.59     13612
      family       1.00      0.00      0.01       783
     fantasy       0.00      0.00      0.00       322
   game-show       0.98      0.22      0.36       193
     history       0.00      0.00      0.00       243
      horror       0.70      0.33      0.45      2204
       music       0.74      0.14      0.24       731
     musical       0.00      0.00      0.00       276
     mystery       0.00      0.00     

c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression - Accuracy: 0.5871586715867159
              precision    recall  f1-score   support

      action       0.46      0.29      0.36      1314
       adult       0.62      0.27      0.38       590
   adventure       0.57      0.18      0.27       775
   animation       0.51      0.08      0.14       498
   biography       0.00      0.00      0.00       264
      comedy       0.54      0.59      0.56      7446
       crime       0.32      0.04      0.07       505
 documentary       0.67      0.85      0.75     13096
       drama       0.55      0.77      0.64     13612
      family       0.49      0.10      0.17       783
     fantasy       0.58      0.06      0.11       322
   game-show       0.83      0.54      0.65       193
     history       0.00      0.00      0.00       243
      horror       0.65      0.58      0.61      2204
       music       0.64      0.45      0.53       731
     musical       0.20      0.01      0.03       276
     mystery       0.38      0

c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\harih\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
best_model = LogisticRegression(max_iter=1000)
best_model.fit(X_train, y_train)
from joblib import dump
dump(best_model, 'movie_genre_classifier.pkl')
dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']